In [1]:
import numpy as np
import pandas as pd
import os
import glob

In [2]:
# High_AUC_PPV_FUNCTIONS = pd.read_csv("High_AUC_PPV_FUNCTIONS.txt", sep="\t")
# High_AUC_PPV_FUNCTIONS

In [3]:
# Top_148_test_union_function_groups = pd.read_csv("Top_148_test_union_function_groups.txt", sep="\t")
# Top_148_test_union_function_groups

### If needed, remember to change the permission of files and folders

## Protein file

In [4]:
def create_protein_filter_list(protein_file_name, protein_cutoff_value):
    protein_df = pd.read_csv(protein_file_name,  names = ["Protein_name","Score"], header=0, index_col=0)
    protein_filter = protein_df[protein_df['Score'] > protein_cutoff_value]
    return protein_filter

### In the protein file, max = 0.25, see the question from email

## IDR file

In [5]:
# https://plotly.com/python/v3/html-reports/#generate-html-reports-with-d3-graphsusing-python-plotly-and-pandas
def create_IDR_filter_list(IDR_file_name, IDR_cutoff_value, filtered_protein_name):
    IDR_df = pd.read_csv(IDR_file_name, names = ["Protein IDR","Score"], header=0)
    
    filter_IDRs = pd.DataFrame()
    for p_name in filtered_protein_name:
        for IDR_name in IDR_df["Protein IDR"].values:
            if p_name in IDR_name:
                s2 = IDR_df[IDR_df["Protein IDR"] == IDR_name]
                filter_IDRs = pd.concat([filter_IDRs, s2], ignore_index=True)

    filter_IDRs = filter_IDRs[filter_IDRs["Score"] > IDR_cutoff_value]
    
    # Split GO_term string in order to let user understand the meaning of each column
    # https://stackoverflow.com/questions/14745022/how-to-split-a-dataframe-string-column-into-two-columns
    # n=the number of columns you expect
    filter_IDRs[['UniProtID', 'Redundancy', 'The beginning position of IDR', 'The ending position of IDR']] = filter_IDRs['Protein IDR'].str.split('_', n=4, expand=True)
    filter_IDRs = filter_IDRs.drop(columns=['Protein IDR', 'Redundancy'])
    filter_IDRs = filter_IDRs[['UniProtID', 'The beginning position of IDR', 'The ending position of IDR', 'Score']]
    
    # hold two digits after the decimal points
    filter_IDRs["Score"] = filter_IDRs["Score"].astype(float)
    filter_IDRs["Score"] = filter_IDRs["Score"].round(2)
    
    # rename "Score"
    filter_IDRs = filter_IDRs.rename(columns={"Score": "The probability of the association between the IDR and the function"})
    
    html_filter_IDRs = filter_IDRs.to_html(index=False).replace('<table border="1" class="dataframe">','<table class="table table-striped">')
    html_filter_IDRs = html_filter_IDRs.replace('<td>', '<td style="text-align: center;">') # center text
    html_filter_IDRs= html_filter_IDRs.replace('<th>', '<th style="text-align: center;">') # center column names
    return html_filter_IDRs 

## Features

In [6]:
def create_feature_filter_list(feature_file_name):
    feature_df = pd.read_csv(feature_file_name, names = ["Feature name","Score"])
    filter_features = feature_df.dropna()
    
    # hold two digits after the decimal points
    filter_features["Score"] = filter_features["Score"].astype(float)
    filter_features["Score"] = filter_features["Score"].round(2)
    
    # rename "Score"
    filter_features = filter_features.rename(columns={"Score": "The strength and direction of association with function for each molecular feature"})    
    
    html_filter_features = filter_features.to_html(index=False).replace('<table border="1" class="dataframe">','<table class="table table-striped">')
    html_filter_features = html_filter_features.replace('<td>', '<td style="text-align: center;">') # center text
    html_filter_features = html_filter_features.replace('<th>', '<th style="text-align: center;">') # center column names
    return html_filter_features

## HTML

In [7]:
# refer to https://plotly.com/python/v3/html-reports/#generate-html-reports-with-d3-graphsusing-python-plotly-and-pandas
def convert_to_HTML_format(filter_IDRs, filter_features, function_name):
    html_string = '''
    <html>
        <head>
            <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.1/css/bootstrap.min.css">
            <style>body{ margin:0 100; background:whitesmoke; }</style>
        </head>
        <body>
            <h1>Group/Function: ''' + function_name + '''</h1>

            <!-- *** Two Lists *** --->
            <h2>IDRs List</h2>
            ''' + filter_IDRs + '''

            <h2>Features List</h2>
            ''' + filter_features + '''
        </body>
    </html>'''

    open_path = f'/home/shuting/IDRs/webs/{function_name}.html' # put a variable within a string
    f = open(open_path,'w')
    f.write(html_string)
    f.close()

# Functions

## Automatically read each csv file in one folder

### There is huge differences between using severl "if" and "if, elif, else"

In [8]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))
print(len(csv_files))
  
num = 0
# loop over the list of csv files
for f in csv_files:
    
    protein_filter = pd.DataFrame([])
    filter_IDRs = pd.DataFrame([])
    filter_features = pd.DataFrame([])
    print("num:\t", num)
    
    # read the first csv file
    if num == 0:
        df_post_filename = f.split("/")[-1]
        num += 1
    
    # read the second csv file
    elif num == 1:
        df_pred_filename = f.split("/")[-1]
        num += 1
        
    # read the third csv file
    else:
        df_stat_filename = f.split("/")[-1]
        num = 0
        
        # so far, read all files of one function
        # print("Group:\t", df_post_filename, df_pred_filename, df_stat_filename, "\t")
        
        # step1: protein
        protein_file_name = df_pred_filename
        protein_cutoff_value = 0.15
        protein_filter = create_protein_filter_list(protein_file_name, protein_cutoff_value)
        filtered_protein_name = protein_filter.index.values
        print ("Protein\n", filtered_protein_name)

        # step2: IDRs
        IDR_file_name = df_post_filename
        IDR_cutoff_value = 0.5
        html_filter_IDRs = create_IDR_filter_list(IDR_file_name, IDR_cutoff_value, filtered_protein_name)
        # print("IDR\n", filter_IDRs)

        # step3: features
        feature_file_name = df_stat_filename
        html_filter_features = create_feature_filter_list(feature_file_name)
        # print("Feature\n", filter_features)

        # step4: HTML
        function_name = df_pred_filename.split()[0]
        function_name = function_name[:-1]
        convert_to_HTML_format(html_filter_IDRs, html_filter_features, function_name)

15
num:	 0
num:	 1
num:	 2
Protein
 ['Q8N9Q2' 'Q8NE71' 'O00567' 'O75167' 'O75475' 'P07305' 'P18583' 'P35659'
 'P46013' 'Q14093' 'Q5SSJ5' 'Q8NFC6' 'Q969V6' 'Q96T23' 'Q9BW71' 'Q9NR30'
 'Q9NTI5' 'O00294' 'Q9Y6X4' 'O15446' 'O60524' 'P22492' 'Q02952' 'Q13127'
 'Q13428' 'Q14978']


/tmp/ipykernel_372/1986022666.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_features["Score"] = filter_features["Score"].astype(float)
/tmp/ipykernel_372/1986022666.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_features["Score"] = filter_features["Score"].round(2)


num:	 0
num:	 1
num:	 2
Protein
 ['Q5VWQ0' 'Q8IUX7' 'Q8N3Z6' 'Q8N9E0' 'Q8NE71' 'Q8WTT2' 'Q8WWL7' 'Q96JP5'
 'Q99549' 'Q99848' 'Q9BQ75' 'Q9BQG0' 'Q9GZR7' 'Q9NP64' 'Q9NPI1' 'Q9NQZ2'
 'Q9NVP1' 'Q9NVU7' 'Q9NZM5' 'Q9P2K1' 'O00567' 'Q9Y6V7' 'O15066' 'O15213'
 'O75475' 'O75683' 'P07305' 'P0C671' 'P15514' 'P20042' 'P35269' 'P35663'
 'P42696' 'P50579' 'Q03468' 'Q13562' 'Q14093' 'Q14690' 'Q15020' 'Q15361'
 'Q2TBE0' 'Q3B726' 'Q3B820' 'Q5SSJ5' 'Q86XH1' 'Q8TDI8' 'Q96BK5' 'Q96T23'
 'Q9BRJ6' 'Q9BRU9' 'Q9BT09' 'Q9BVP2' 'Q9H1E3' 'Q9H501' 'Q9H6F5' 'Q9NQS7'
 'Q9NR30' 'Q9P031' 'Q9Y2X3' 'Q6PCB5' 'Q6PII3' 'O00294' 'O00541' 'O14617'
 'O15446' 'O43159' 'O60524' 'O75151' 'O76021' 'P11387' 'P12272' 'P22492'
 'P27816' 'Q6UX04' 'P82970' 'Q13127' 'Q13601' 'Q13823' 'Q14152' 'Q14978'
 'Q15397' 'Q1ED39' 'Q5BKY9' 'Q5C9Z4' 'Q5JTH9' 'Q5JU67' 'Q8N157' 'Q8N9T8'
 'Q8NEJ9']


/tmp/ipykernel_372/1986022666.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_features["Score"] = filter_features["Score"].astype(float)
/tmp/ipykernel_372/1986022666.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_features["Score"] = filter_features["Score"].round(2)
/tmp/ipykernel_372/1986022666.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

num:	 0
num:	 1
num:	 2
Protein
 ['Q9Y493' 'P05997' 'P08572' 'P12107' 'P39059' 'P39060' 'P53420' 'Q5TAT6'
 'P02458' 'P02462' 'P08123' 'P20849' 'P25067' 'P27658' 'P29400' 'Q6UWP8'
 'Q01955' 'Q02388' 'Q03692' 'Q13444' 'Q14031' 'Q14055' 'Q14242' 'Q14993']
num:	 0
num:	 1
num:	 2
Protein
 ['Q9Y493' 'P05997' 'P08572' 'P12107' 'P39059' 'P39060' 'P53420' 'Q5TAT6'
 'P02458' 'P02462' 'P08123' 'P20849' 'P25067' 'P27658' 'P29400' 'Q6UWP8'
 'Q01955' 'Q02388' 'Q03692' 'Q13444' 'Q14031' 'Q14055' 'Q14242' 'Q14993']
num:	 0
num:	 1
num:	 2
Protein
 ['Q92794' 'Q9NU22' 'Q9NVP1' 'P35659' 'P49711' 'Q96T23' 'O00294' 'O60524'
 'P78362']


/tmp/ipykernel_372/1986022666.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_features["Score"] = filter_features["Score"].astype(float)
/tmp/ipykernel_372/1986022666.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_features["Score"] = filter_features["Score"].round(2)
/tmp/ipykernel_372/1986022666.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation